In [10]:
import ast
import re
from tqdm import tqdm
from parsing import get_inline_citations
from utils import load_dataset

In [2]:
astro_reviews = load_dataset("data/sentence_segmented/Astro_Reviews.json")
citations = get_inline_citations(astro_reviews[0]['body'])
print(len(citations))

data/sentence_segmented/Astro_Reviews.json: 996/996 have all required keys
352


In [6]:
import os
import PyPDF2
import requests
from pprint import pprint


def ask_ollama(prompt):
    url = "http://localhost:11434/api/generate"
    payload = {
        "model": "llama3.3",
        "prompt": prompt,
        "temperature": 0.01,
        "stream": False
    }

    response = requests.post(url, json=payload)
    result = response.json()
    return result['response']

In [21]:
prompt = """
Extract and output ONLY the inline citations from the text below as a list of tuples
- Each citation becomes a (string, int) tuple where the string is the first author's name and the int is the year
- If there are no citations in the text, output []
- Do not count citations 'in preparation' or lacking a year
- Do not include any introductory text, explanations, or anything before or after the array

Examples of inline citations:
'''
Sentence: "Like Caffau et al. (2008a) , we have similar findings."
Output: [('Caffau et al.', 2008)]

Sentence: "Methods for mixing below the convection zone are well understood ( Brun, Turck-Chièze Zahn 1999 , Charbonnel Talon 2005 )."
Output: [('Brun', 1999), ('Charbonnel', 2005)]

Sentence: "Momentum balance gives an expression ( Fabian 1999 ; Fabian, Wilman Crawford 2002 ; King 2003 , 2005 )"
Output: [('Fabian', 1999), ('Fabian', 2002), ('King', 2003), ('King', 2005)]

Sentence: "This is consistent with previous results (Pereira et al., in preparation)."
Output: []
'''

Now extract the inline citations from the following text:
'''
{text}
'''

Output format: 
[('first author', year), ('first author', year), ...]
"""
text = 'neglect the H collisions altogether based on the available atomic physics data for other elements, while others use the classical Drawin (1968) formula, possibly with a scaling factor S H that typically varies from 0 to 1. Holweger (2001) found log ε O = 8.71 ± 0.05 using the Holweger Müller (1974) model with granulation corrections'

# print(prompt.format(text=text))
response = ask_ollama(prompt.format(text=text))
print(response)

 [('Drawin', 1968), ('Holweger', 2001), ('Holweger', 1974)]


In [19]:
LIST_PATTERN = re.compile(r"\[.*?\]")

def response_to_list(response):
    match = re.search(LIST_PATTERN, response)
    if not match:
        print("No match found:", response)
        return []
    try:
        lst = ast.literal_eval(match.group())
        return lst
    except Exception as e:
        with open('unparsed_citations.txt', 'a') as f:
            f.write(match.group() + "\n")
        return []

postprocess_response = response_to_list(response)
print(type(postprocess_response))
print(postprocess_response)

<class 'list'>
[('Drawin', 1968), ('Holweger', 2001), ('Holweger Müller', 1974)]


In [13]:
citations = []
for sentence in tqdm(astro_reviews[0]['body_sentences']):
    new_citations = response_to_list(ask_ollama(prompt.format(text=sentence)))
    print(f"Sentence: {sentence}")
    print(f"Citations: {new_citations}")
    print("=====")
    citations.extend(new_citations)

print(len(citations))

 10%|█         | 1/10 [00:02<00:23,  2.61s/it]

Sentence: 1. INTRODUCTION It has been realized over the past decade that the black hole at the center of a galaxy bulge is no mere ornament but may play a major role in determining the final stellar mass of the bulge. 
Citations: []
=====


 20%|██        | 2/10 [00:04<00:17,  2.13s/it]

Sentence: The process by which this occurs is known as AGN (active galactic nuclei) feedback, and it takes place through an interaction between the energy and radiation generated by accretion onto the massive black hole (the active nucleus) and the gas in the host galaxy. 
Citations: []
=====


 30%|███       | 3/10 [00:06<00:13,  1.98s/it]

...

Sentence: The first is the radiative mode, also known as the quasar or wind mode, which operates, or operated, in a typical bulge when the accreting black hole was close to the Eddington limit. 
Citations: []
=====


 90%|█████████ | 9/10 [00:14<00:01,  1.48s/it]

Sentence: It is most concerned with pushing cold gas about. 
Citations: []
=====


100%|██████████| 10/10 [00:15<00:00,  1.60s/it]

Sentence: The second mode is the kinetic mode, also known as the radio jet or maintenance mode. 
Citations: []
=====
0


In [22]:
print(len(citations))

326


In [26]:
for i, citation in enumerate(citations):
    print(citation)

('Kormendy', 2001)
('Merritt', 2001)
('Häring', 2004)
('Jahnke', 2011)
('Kauffmann', 2000)
('Granato', 2004)
('Di Matteo', 2005)
('Springel', 2005)
('Bower', 2006)
('Croton', 2006)
('Hopkins', 2006)
('Ciotti', 2010)
('Scannapieco', 2012)
('Silk Rees', 1998)
('Haehnelt, Natarajan Rees', 1998)
('Fabian', 1999)
('Fabian', 2002)
('King', 2003)
('King', 2005)
('Murray', 2005)
('Gültekin', 2009)
('Kollmeier', 2006)
('Steinhardt Elvis', 2010)
('King', 2003)
('Laor', 1993)
('Scoville', 1995)
('Murray', 2005)
('Fabian', 2008)
('Raimundo', 2010)
('Faber Jackson', 1976)
('Djorgovski', 1987)
('Faber', 1987)
('Sanders McGaugh', 2002)
('Silk Nusser', 2010)
('van Dokkum', 2010)
('Blandford Payne', 1982)
('Reynolds', 1997)
('Crenshaw', 2003)
('Blustin', 2005)
('Weymann', 1991)
('Ganguly', 2007)
('Pounds', 2003)
('Reeves', 2009)
('Tombesi', 2010)
('Tombesi', 2012)
('Tombesi', 2010)
('Reeves', 2009)
('Rupke Veilleux', 2011)
('Dunn', 2010)
('Moe', 2009)
('Saez', 2009)
('Heckman', 2000)
('Veilleux', 2005)

In [20]:
citations = []
for sentence in tqdm(astro_reviews[0]['body_sentences']):
    new_citations = response_to_list(ask_ollama(prompt.format(text=sentence)))
    print(f"Sentence: {sentence}")
    print(f"Citations: {new_citations}")
    print("=====")
    citations.extend(new_citations)
print(len(citations))

  0%|          | 1/600 [00:04<43:22,  4.34s/it]

Sentence: 1. INTRODUCTION It has been realized over the past decade that the black hole at the center of a galaxy bulge is no mere ornament but may play a major role in determining the final stellar mass of the bulge. 
Citations: []
=====


  0%|          | 2/600 [00:06<28:21,  2.85s/it]

Sentence: The process by which this occurs is known as AGN (active galactic nuclei) feedback, and it takes place through an interaction between the energy and radiation generated by accretion onto the massive black hole (the active nucleus) and the gas in the host galaxy. 
Citations: []
=====


  0%|          | 3/600 [00:07<23:29,  2.36s/it]

...

Sentence: This has profound implications for our understanding of galaxy, group, and cluster evolution and has ramifications for precision cosmology using galaxies ( van Daalen et al. 2011 ). 
Citations: [('van Daalen', 2011)]
=====


100%|█████████▉| 599/600 [1:28:50<00:01,  1.86s/it]

Sentence: AGN feedback appears to be an important aspect in the complexity of the baryonic Universe. 
Citations: []
=====


100%|██████████| 600/600 [1:28:51<00:00,  8.89s/it]

Sentence: Table 1 Observational evidence for AGN feedback
Citations: []
=====
326


In [18]:
citations

[('Kormendy', 2001),
 ('Merritt', 2001),
 ('Häring', 2004),
 ('Jahnke', 2011),
 ('Kauffmann Haehnelt', 2000),
 ('Granato', 2004),
 ('Di Matteo', 2005),
 ('Springel', 2005),
 ('Bower', 2006),
 ('Croton', 2006),
 ('Hopkins', 2006),
 ('Ciotti', 2010),
 ('Scannapieco', 2012),
 ('Silk Rees', 1998),
 ('Haehnelt, Natarajan Rees', 1998),
 ('Fabian', 1999),
 ('Fabian', 2002),
 ('King', 2003),
 ('King', 2005),
 ('Murray', 2005),
 ('Gültekin', 2009),
 ('Kollmeier', 2006),
 ('Steinhardt Elvis', 2010),
 ('King', 2003),
 ('Laor Draine', 1993),
 ('Scoville Norman', 1995),
 ('Murray, Quartaert Thompson', 2005),
 ('Fabian', 2008),
 ('Raimundo', 2010),
 ('Faber Jackson', 1976),
 ('Djorgovski', 1987),
 ('Faber', 1987),
 ('Sanders McGaugh', 2002),
 ('Silk Nusser', 2010),
 ('van Dokkum', 2010),
 ('Blandford Payne', 1982),
 ('Reynolds', 1997),
 ('Crenshaw', 2003),
 ('Blustin', 2005),
 ('Weymann', 1991),
 ('Ganguly', 2007),
 ('Pounds', 2003),
 ('Reeves', 2009),
 ('Tombesi', 2010),
 ('Tombesi', 2012),
 ('Tomb